<a href="https://colab.research.google.com/github/Chubbyman2/Resume/blob/master/AlexNet_Paper_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Implementation of this paper:

https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf

Colaboratory System Requirements:
*   25GB version of colab
*   TPU Acceleration

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import time

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator # For image augmentation

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 6s 0us/step


In [3]:
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [4]:
# CIFAR dataset has 50,000 training samples and 10,000 test samples
validation_images, validation_labels = train_images[:5000], train_labels[:5000]
train_images, train_labels = train_images[5000:], train_labels[5000:]

In [5]:
# Takes the train, test, val datasets
# Returns corresponding tf Dataset representations
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
validation_ds = tf.data.Dataset.from_tensor_slices((validation_images, validation_labels))

In [6]:
BATCH_SIZE = 128
IMG_SHAPE = 224

def process_images(image, label):
    # Normalize images to have a mean of 0 and standard deviation of 1
    image = tf.image.per_image_standardization(image)

    # Resize images from 32x32 to 224x224
    image = tf.image.resize(image, (IMG_SHAPE, IMG_SHAPE))

    return image, label

In [7]:
# Get size of each dataset
train_ds_size = tf.data.experimental.cardinality(train_ds).numpy()
test_ds_size = tf.data.experimental.cardinality(test_ds).numpy()
validation_ds_size = tf.data.experimental.cardinality(validation_ds).numpy()

In [8]:
# Preprocess data, shuffle dataset, put data into batches
train_ds = (train_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=BATCH_SIZE, drop_remainder=True))
test_ds = (test_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=BATCH_SIZE, drop_remainder=True))
validation_ds = (validation_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=BATCH_SIZE, drop_remainder=True))

Note: The paper details a technique not available in Tensorflow named "overlapping pooling".

Tensorflow mandates that kernel sizes be that of *z* x *z*, whereas overlapping pooling uses kernel sizes of 2 x 3.

In [9]:
# Build model
AlexNet = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=96, input_shape=(IMG_SHAPE, IMG_SHAPE, 3), kernel_size=(11, 11), strides=(4, 4), activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(2, 2),

    tf.keras.layers.Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), activation="relu", padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(2, 2),

    tf.keras.layers.Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation="relu", padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(2, 2),

    tf.keras.layers.Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation="relu", padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(2, 2),

    tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation="relu", padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(2, 2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4096, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4096, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation="softmax")
])

In [10]:
# Compile model
optimizer = tf.keras.optimizers.SGD(lr=0.01, momentum=0.9)
AlexNet.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [11]:
AlexNet.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 54, 54, 96)        34944     
_________________________________________________________________
batch_normalization (BatchNo (None, 54, 54, 96)        384       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 27, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 384)       8

In [12]:
# Configure checkpoints
checkpoint_dir = "./training_checkpoints"

# Name of checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

In [13]:
# Train model
EPOCHS = 90

# The original num of epochs was 90
# but after ~9 epochs, model begins to overfit
# Early stopping stops model after 1 epoch if val_acc drops
early_stopping = tf.keras.callbacks.EarlyStopping(patience=1)

history = AlexNet.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=validation_ds,
    callbacks=[early_stopping, checkpoint_callback]
)

Epoch 1/90
351/351 [==============================] - 489s 1s/step - loss: 1.5246 - accuracy: 0.4514 - val_loss: 1.3184 - val_accuracy: 0.5266
Epoch 2/90
351/351 [==============================] - 487s 1s/step - loss: 1.0853 - accuracy: 0.6162 - val_loss: 1.1386 - val_accuracy: 0.5974
Epoch 3/90
351/351 [==============================] - 485s 1s/step - loss: 0.8724 - accuracy: 0.6912 - val_loss: 0.9620 - val_accuracy: 0.6725
Epoch 4/90
351/351 [==============================] - 481s 1s/step - loss: 0.6985 - accuracy: 0.7560 - val_loss: 0.9405 - val_accuracy: 0.6827
Epoch 5/90
351/351 [==============================] - 483s 1s/step - loss: 0.5568 - accuracy: 0.8056 - val_loss: 0.8804 - val_accuracy: 0.7079
Epoch 6/90
351/351 [==============================] - 494s 1s/step - loss: 0.4156 - accuracy: 0.8557 - val_loss: 0.9414 - val_accuracy: 0.7065


In [14]:
# Final test
AlexNet.evaluate(test_ds)

78/78 [==============================] - 18s 236ms/step - loss: 0.9886 - accuracy: 0.6959


[0.9885995388031006, 0.6959134340286255]